In [1]:
# --------------------------------------------
# 0️⃣ Setup: Install dependencies
# --------------------------------------------
!pip install retina-face opencv-python


In [3]:
import cv2
import os
from retinaface import RetinaFace
import shutil
from google.colab import files

# --------------------------------------------
# 1️⃣ Upload video from local to Colab
# --------------------------------------------
uploaded = files.upload()  # Allow user to upload the video
video_path = list(uploaded.keys())[0]  # Use the first uploaded file

# Create a directory to save cropped faces
output_dir = 'cropped_faces'
os.makedirs(output_dir, exist_ok=True)

# --------------------------------------------
# 2️⃣ Read the video
# --------------------------------------------
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise FileNotFoundError(f"Video file {video_path} not found.")

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration_seconds = frame_count / fps

def get_frame_at_timestamp(timestamp_sec):
    cap.set(cv2.CAP_PROP_POS_MSEC, timestamp_sec * 1000)
    ret, frame = cap.read()
    return frame if ret else None

# --------------------------------------------
# 3️⃣ Process 10s clips every 5 minutes
# --------------------------------------------
clip_duration = 10
interval = 5 * 60

for start_time in range(0, int(duration_seconds), interval):
    end_time = start_time + clip_duration
    print(f"\n🎞️ Processing clip: {start_time}s to {end_time}s")

    for timestamp in range(start_time, end_time):
        frame = get_frame_at_timestamp(timestamp)
        if frame is None:
            print(f"⚠️ No frame at {timestamp}s.")
            continue

        # 4️⃣ Face Detection
        try:
            faces = RetinaFace.detect_faces(frame)
        except Exception as e:
            print(f"❌ Face detection failed at {timestamp}s: {e}")
            continue

        if not faces:
            print(f"😐 No faces detected at {timestamp}s.")
            continue

        clip_dir = os.path.join(output_dir, f"clip_{start_time}-{end_time}s")
        os.makedirs(clip_dir, exist_ok=True)

        for i, (key, face) in enumerate(faces.items()):
            x1, y1, x2, y2 = face["facial_area"]
            face_img = frame[y1:y2, x1:x2]
            face_filename = os.path.join(clip_dir, f"face_{timestamp}_{i+1}.jpg")
            cv2.imwrite(face_filename, face_img)
            print(f"✅ Saved: {face_filename}")

# --------------------------------------------
# 5️⃣ Compress and Download Results
# --------------------------------------------
zip_path = shutil.make_archive(output_dir, 'zip', output_dir)
print(f"\n📦 Cropped faces saved to ZIP: {zip_path}")

# Download to local machine
files.download(zip_path)


Saving 172.16.35.38_IT LAB 2(1)_main_20250718122459.dav to 172.16.35.38_IT LAB 2(1)_main_20250718122459.dav

🎞️ Processing clip: 0s to 10s
✅ Saved: cropped_faces/clip_0-10s/face_0_1.jpg
✅ Saved: cropped_faces/clip_0-10s/face_0_2.jpg
✅ Saved: cropped_faces/clip_0-10s/face_0_3.jpg
✅ Saved: cropped_faces/clip_0-10s/face_0_4.jpg
✅ Saved: cropped_faces/clip_0-10s/face_0_5.jpg
✅ Saved: cropped_faces/clip_0-10s/face_0_6.jpg
✅ Saved: cropped_faces/clip_0-10s/face_0_7.jpg
✅ Saved: cropped_faces/clip_0-10s/face_0_8.jpg
✅ Saved: cropped_faces/clip_0-10s/face_0_9.jpg
✅ Saved: cropped_faces/clip_0-10s/face_0_10.jpg
✅ Saved: cropped_faces/clip_0-10s/face_1_1.jpg
✅ Saved: cropped_faces/clip_0-10s/face_1_2.jpg
✅ Saved: cropped_faces/clip_0-10s/face_1_3.jpg
✅ Saved: cropped_faces/clip_0-10s/face_1_4.jpg
✅ Saved: cropped_faces/clip_0-10s/face_1_5.jpg
✅ Saved: cropped_faces/clip_0-10s/face_1_6.jpg
✅ Saved: cropped_faces/clip_0-10s/face_1_7.jpg
✅ Saved: cropped_faces/clip_0-10s/face_1_8.jpg
✅ Saved: cropp

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
import shutil

# Path to your ZIP file
zip_path = "/content/cropped_faces.zip"
extract_path = "/content/cropped_faces"

# Unzip
shutil.unpack_archive(zip_path, extract_path)

print("✅ Unzipped successfully to:", extract_path)


✅ Unzipped successfully to: /content/cropped_faces


In [4]:
import cv2
import os
import shutil
from google.colab import files

# Input & output directories
input_dir = "cropped_faces"
output_dir = "arcface_faces"

# Clean previous arcface_faces if exists
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# Walk through each clip folder
for clip_folder in os.listdir(input_dir):
    clip_path = os.path.join(input_dir, clip_folder)
    if not os.path.isdir(clip_path):
        continue

    # Create same subfolder in arcface_faces
    save_clip_path = os.path.join(output_dir, clip_folder)
    os.makedirs(save_clip_path, exist_ok=True)

    # Process each face image
    for file in os.listdir(clip_path):
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            file_path = os.path.join(clip_path, file)

            # Read image (BGR)
            img = cv2.imread(file_path)
            if img is None:
                print(f"⚠️ Skipping {file_path}, cannot read.")
                continue

            # Convert BGR → RGB
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Resize to (112,112)
            img_resized = cv2.resize(img_rgb, (112, 112))

            # Save processed face in arcface_faces/ with same structure
            save_path = os.path.join(save_clip_path, file)
            cv2.imwrite(save_path, cv2.cvtColor(img_resized, cv2.COLOR_RGB2BGR))

print("✅ All faces resized to 112x112 and saved in arcface_faces/ with same clip structure")

# --------------------------------------------
# 🔹 Make a ZIP file
# --------------------------------------------
zip_path = shutil.make_archive(output_dir, 'zip', output_dir)
print(f"📦 ArcFace-ready dataset zipped at: {zip_path}")

files.download(zip_path)


✅ All faces resized to 112x112 and saved in arcface_faces/ with same clip structure
📦 ArcFace-ready dataset zipped at: /content/arcface_faces.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>